In [1]:
# %pip install opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 8.1 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import json
import sys
import numpy as np

In [8]:
image_path = "/Users/jerome/Code/legion/client/assets/tiles.png"
new_json_file_path = "/Users/jerome/Code/legion/client/assets/tiles2.json"
json_file = "/Users/jerome/Code/legion/client/assets/tiles.json"
debug_image_path = "/Users/jerome/Code/legion/client/assets/tiles_debug.png"
transparent_image_path = "/Users/jerome/Code/legion/client/assets/tiles_tp.png"


In [7]:

# Image file path

# Read the image
image = cv2.imread(image_path)
if image is None:
    sys.exit("Could not read the image.")

# Convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Thresholding the image
_, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

# Finding contours in the image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initializing the JSON structure for Phaser 3 atlas
atlas_json = {
    "frames": {}
}

# Iterate over the found contours
for i, cnt in enumerate(contours):
    x, y, w, h = cv2.boundingRect(cnt)
    # Considering each contour where width and height are more than 10, avoiding noise
    if w > 100 and h > 100:
        frame_name = f"element_{i}"
        atlas_json["frames"][frame_name] = {
            "frame": {"x": x, "y": y, "w": w, "h": h},
            "rotated": False,
            "trimmed": False,
            "spriteSourceSize": {"x": 0, "y": 0, "w": w, "h": h},
            "sourceSize": {"w": w, "h": h},
        }
        # Drawing a green rectangle around the detected square
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Writing JSON to a file
with open(json_file, 'w') as jf:
    json.dump(atlas_json, jf)

# Save the modified image
cv2.imwrite(debug_image_path, image)

print(f"JSON atlas generated at {json_file}")
print(f"Debug image with contours drawn is saved at {debug_image_path}")


JSON atlas generated at /Users/jerome/Code/legion/client/assets/tiles.json
Debug image with contours drawn is saved at /Users/jerome/Code/legion/client/assets/tiles_debug.png


In [6]:

# Convert the image to the BGRA format
image_bgra = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

# Define the white threshold
lower_white = np.array([220, 220, 220, 255])
upper_white = np.array([255, 255, 255, 255])

# Create a mask for the white background
white_mask = cv2.inRange(image_bgra, lower_white, upper_white)

# Set the alpha channel to 0 where the white background is
image_bgra[white_mask == 255, 3] = 0

# Save the modified image
cv2.imwrite(transparent_image_path, image_bgra)

print(f"Image with transparent background is saved at {transparent_image_path}")

# Continue with the rest of the original script...

Image with transparent background is saved at /Users/jerome/Code/legion/client/assets/tiles_tp.png


In [9]:
import json

# Load the atlas JSON
json_file_path = json_file
with open(json_file_path, 'r') as f:
    atlas_json = json.load(f)

# Extract frames and sort them
frames = list(atlas_json['frames'].items())
frames.sort(key=lambda item: (item[1]['frame']['y'], item[1]['frame']['x']))

# Create a new ordered JSON object
new_atlas_json = {"frames": {}}

# Populate the new JSON object with sorted frames
for frame_name, frame_data in frames:
    new_atlas_json['frames'][frame_name] = frame_data

# Save the reordered atlas JSON
with open(new_json_file_path, 'w') as f:
    json.dump(new_atlas_json, f)

print(f"Reordered JSON atlas generated at {new_json_file_path}")


Reordered JSON atlas generated at /Users/jerome/Code/legion/client/assets/tiles2.json
